In [ ]:
%pip install -Uqq fastbook

In [ ]:
from fastbook import *

In [ ]:
urls = search_images_ddg('bird photos', max_images=1)
len(urls), urls[0]

In [ ]:
import time
filename = f"bird_{time.time_ns()}.jpg"
dest = Path(filename)
if not dest.exists(): download_url(urls[0], dest, show_progress=False)
dest

In [ ]:
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
searches = 'forest', 'bird'
path = Path('bird_or_not')

# Always create base directory
path.mkdir(exist_ok=True, parents=True)

for o in searches:
    class_path = path/o  # bird_or_not/forest or bird_or_not/bird
    
    # Create class directory if missing
    class_path.mkdir(exist_ok=True, parents=True)
    
    # Only download if directory is empty
    if len(list(class_path.glob('*'))) == 0:
        print(f"Downloading {o} images...")
        results = search_images_ddg(f'{o} photo', max_images=200)
        download_images(class_path, urls=results)
        resize_images(class_path, max_size=400, dest=class_path)

In [ ]:
print("\nFinal directory status:")
for o in searches:
    class_path = path/o
    imgs = get_image_files(class_path)
    print(f"{o}: {len(imgs)} images")
    if len(imgs) > 0:
        print(f"  Sample: {imgs[0]}")

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

In [ ]:
# Check total images
image_files = get_image_files(path)
print(f"Total images found: {len(image_files)}")

# Check per category
for category in searches:
    cat_files = get_image_files(path/category)
    print(f"{category} images: {len(cat_files)}")

In [ ]:
# Sample first 5 image paths
print("Sample image paths:")
for f in image_files[:5]:
    print(f)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create(dest))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]: .4f}")